In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod 
import xgi ## pip install xgi is required
import pickle
from collections import Counter
from functools import reduce
import itertools
from scipy.special import comb
import warnings
import random
from itertools import combinations
from itertools import combinations as combs

## Functions to compute various simpliciality measures are found in the included ```simpliciality.py``` file:
import simpliciality as spl 
import sr ## functions taken from Jordan's notebook

## Set this to the data directory
datadir='../Datasets/'


# XGI's hypergraphs



In [2]:
def compute_all(fn=""):
    H = xgi.load_xgi_data(fn)
    E = list(set([tuple(sorted(e)) for e in H.edges.members() if len(e)<=11 and len(e)>=2]))
    E = [set(e) for e in E]
    H = hnx.Hypergraph(dict(enumerate(E)))
    V = [x for x in H.nodes]
    print('\nResults for',fn,':')
    print('n =',len(V),'\nm =',len(E),'\n<s> =',np.mean([len(e) for e in E]))
    print('SF:',spl.get_simplicial_fraction(V,E),
          '\nES:',spl.get_edit_simpliciality(V,E),
          '\nFES with max face:',spl.get_face_edit_simpliciality(V,E,exclude_self=False),
          '\nFES excluding max face:',spl.get_face_edit_simpliciality(V,E,exclude_self=True))
    print('simplicial ratio with CL model:',spl.get_simplicial_ratio(V, E, samples=1000, multisets=False))
    r, _a, _b, _c, _d = sr.simplicial_ratio(V,E)
    print('simplicial ratio with estimation:',r)
    

In [3]:
def compute_all_time(fn):

    ## load dataset
    H = xgi.load_xgi_data(fn)

    ## check if edges are time-ordered
    TS = [H.edges[e]['timestamp'] for e in H.edges]
    order = [i for i in np.argsort(TS)]
    edges = [e for e in H.edges]
    M = H.edges.members()
    #[(tuple(sorted(M[i])),H.edges[edges[i]]['timestamp']) for i in order]
    L = [tuple(sorted(M[i])) for i in order]

    ## keep only first instance for each edge
    seen = set()
    uniq = []
    for x in L:
        if x not in seen:
            if len(x)<=11 and len(x)>=2:
                uniq.append(x)
            seen.add(x)
    E = [set(e) for e in uniq]

    ## compute SR's
    H = hnx.Hypergraph(dict(enumerate(E)))
    V = [x for x in H.nodes]
    print(fn)
    print('simplicial ratio (CL):',spl.get_simplicial_ratio(V, E, samples=1000, multisets=False))
    r, _a, _b, _c, _d = sr.simplicial_ratio(V,E)
    print('simplicial ratios (estimate):',r,_a,_b)


In [ ]:
Datasets = ["contact-primary-school", "contact-high-school", "hospital-lyon", 
            "email-enron", "email-eu",
            "congress-bills", "tags-ask-ubuntu"]

for fn in Datasets:
    compute_all_time(fn)

contact-primary-school
simplicial ratio (CL): 2.699620513249539
simplicial ratios (estimate): 2.704951861728687 4.75432986030861 0.6555738631487639
contact-high-school
simplicial ratio (CL): 6.550454251155732
simplicial ratios (estimate): 6.582347933788044 11.02970112098354 2.134994746592546
hospital-lyon
simplicial ratio (CL): 0.946515279595781
simplicial ratios (estimate): 0.9624024672023855 1.7535917312247695 0.17121320318000158
email-enron
simplicial ratio (CL): 4.849844288960844
simplicial ratios (estimate): 4.914369981429364 6.913304924798355 2.9154350380603726
email-eu
simplicial ratio (CL): 5.126736017336242
simplicial ratios (estimate): 5.232773578719843 7.8370474662771725 2.6284996911625136
congress-bills
simplicial ratio (CL): 4.4057565721567515
simplicial ratios (estimate): 4.457089538360248 5.227786007020628 3.686393069699869
tags-ask-ubuntu
